In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers
import os
from datetime import datetime
import numpy as np
from openpyxl.styles import PatternFill


light_green = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # jasna zieleń
dark_green = PatternFill(start_color="006100", end_color="006100", fill_type="solid")   # ciemna zieleń


okres_id = '2'
categories = ['CRYPTO', 'CRYPTO NEW']



simulation_data='output_simulations'

#sheet_output='Simulations_summaries'

start_date = '2025-01-01'
start_date = '2024-01-01'
start_date = '2024-01-01'
start_date="2025-01-01"
end_date = '2025-03-01'
sheet_output=f'Simulations_summaries- {start_date}.xlsx'

#strategy='0.4'
intervals=['1w', '3d','2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']
summary_df=pd.DataFrame()
summary_df = pd.read_excel(sheet_output)
    #summary_df = summary_df[summary_df['Strategy'] != 0.3] # usuwam strategy =- 0.3
df = summary_df
#summary_df.to_excel(f'{sheet_output}.xlsx', index=False)


summary_df = df[df['Category'].isin(categories)]
summary_df.head()

df=summary_df
unique_tickers = summary_df['Ticker'].unique()
unique_intervals = summary_df['Interval'].unique()

# --- Tabela przestawna ---
grouped = summary_df.groupby(['Strategy', 'Interval']).agg(
    Count=('P&L after fees', 'count'),
    Sum_Deposit=('Deposit', 'sum'),
    Sum_PnL=('P&L after fees', 'sum'),
    Sum_Number=('Number', 'sum')
).reset_index()

# Oblicz P&L / Deposit
grouped['PnL_per_Deposit'] = grouped['Sum_PnL'] / grouped['Sum_Deposit']

# Tabela przestawna wielopoziomowa
# pivot_df = pd.pivot_table(
#     grouped,
#     index='Strategy',
#     columns='Interval',
#     values=[ 'PnL_per_Deposit'],
#     aggfunc='first',
#     fill_value=0
# )

pivot_df = pd.pivot_table(
    summary_df,
    values='P&L after fees',
    index='Strategy',
    columns='Interval',
    aggfunc='sum',
    fill_value=0,
    margins=True,              # Dodaje sumy
    margins_name='Total'       # Nazwa dla sumy (domyślnie 'All')
)
desired_order = intervals + ['Total'] if 'Total' in pivot_df.columns else intervals

# Przefiltrowanie tylko dostępnych kolumn (niektóre interwały mogą być brakujące)
existing_columns = [col for col in desired_order if col in pivot_df.columns]

# Przestawienie kolejności kolumn
pivot_df = pivot_df[existing_columns]



# Zapis do Excela z nagłówkami
wb = Workbook()
ws = wb.active
ws.title = "Statystyki Strategii"

# Dodanie nagłówka
naglowek = f"Start date: {start_date} | End date: {end_date} | Okres ID: {okres_id}  "

deposit_value = summary_df['Deposit'].iloc[0]
interval_strategy_deposit=deposit_value * len(unique_tickers) 
strategy_deposit =  interval_strategy_deposit * len(unique_intervals)
# Zapis jako jeden wiersz w jednej komórce
ws.append([naglowek])
naglowek = f"  Categories: {', '.join(categories)} |  Num_tickers: {len(unique_tickers)} | Deposit for strategy: {strategy_deposit}  | Deposit for interval: {interval_strategy_deposit} "

# Zapis jako jeden wiersz w jednej komórce
ws.append([naglowek])

naglowek = f"  Suma P&L after fees "

# Zapis jako jeden wiersz w jednej komórce
ws.append([naglowek])


pivot_df_rounded = pivot_df.round(0).astype(int)
pivot_data = pivot_df_rounded.reset_index()
first_strategy = pivot_df_rounded.index[0]

# Dodanie tabeli
for i, row in enumerate(dataframe_to_rows(pivot_data, index=False, header=True)):
    ws.append(row)

    if i == 0:
        continue  # pomijamy nagłówek

    current_row = ws.max_row
    strategy_name = row[0]

    for j, cell in enumerate(ws[current_row][1:], start=1):  # pomijamy kolumnę 'Strategy'
        value = cell.value
        interval = pivot_df_rounded.columns[j - 1]  # dopasowanie kolumny z DF

        if isinstance(value, (int, float)):
            if value > 0:
                cell.fill = light_green

                # Pobierz wartość dla tej kolumny z pierwszej strategii w oryginalnym DF
                first_strategy_value = pivot_df_rounded.loc[first_strategy, interval]

                if strategy_name != first_strategy and value > first_strategy_value:
                    cell.fill = dark_green


# Zapisz plik
excel_path = f"Tabela_Strategii_Szczegoly - {start_date}.xlsx"
wb.save(excel_path)

print(f"Zapisano do pliku: {excel_path}")


print(strategy_deposit)

Zapisano do pliku: Tabela_Strategii_Szczegoly - 2023-08-01.xlsx
237600
